<a href="https://colab.research.google.com/github/u-t-k-a-n/Detection-of-Similar-Points-in-Traffic-Characteristics/blob/main/Model_1_2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = "drive/MyDrive/2017_v1"
files = list()
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("drive/MyDrive/2017_concat/Concat_2017_v2_final.csv")

In [ ]:
df

,speed__abs_energy,speed__absolute_maximum,speed__absolute_sum_of_changes,"speed__agg_autocorrelation__f_agg_""mean""__maxlag_40","speed__agg_autocorrelation__f_agg_""median""__maxlag_40","speed__agg_autocorrelation__f_agg_""var""__maxlag_40","speed__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""max""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""min""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""var""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""max""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""min""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""mean""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""var""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""max""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""min""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""mean""","speed__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""var""","speed__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","speed__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","speed__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","speed__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""","speed__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","speed__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","speed__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","speed__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","speed__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""","speed__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""min""","speed__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""","speed__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""var""","speed__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","speed__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min""","speed__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","speed__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""var""","speed__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""max""","speed__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""","speed__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""mean""","speed__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""var""","speed__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""max""","speed__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""",...,speed__ratio_beyond_r_sigma__r_10,speed__ratio_value_number_to_time_series_length,speed__root_mean_square,speed__skewness,speed__spkt_welch_density__coeff_2,speed__spkt_welch_density__coeff_5,speed__spkt_welch_density__coeff_8,speed__standard_deviation,speed__sum_of_reoccurring_data_points,speed__sum_of_reoccurring_values,speed__sum_values,speed__symmetry_looking__r_0.0,speed__symmetry_looking__r_0.05,speed__symmetry_looking__r_0.1,speed__symmetry_looking__r_0.15000000000000002,speed__symmetry_looking__r_0.2,speed__symmetry_looking__r_0.25,speed__symmetry_looking__r_0.30000000000000004,speed__symmetry_looking__r_0.35000000000000003,speed__symmetry_looking__r_0.4,speed__symmetry_looking__r_0.45,speed__symmetry_looking__r_0.5,speed__symmetry_looking__r_0.55,speed__symmetry_looking__r_0.6000000000000001,speed__symmetry_looking__r_0.65,speed__symmetry_looking__r_0.7000000000000001,speed__symmetry_looking__r_0.75,speed__symmetry_looking__r_0.8,speed__symmetry_looking__r_0.8500000000000001,speed__symmetry_looking__r_0.9,speed__symmetry_looking__r_0.9500000000000001,speed__time_reversal_asymmetry_statistic__lag_1,speed__time_reversal_asymmetry_statistic__lag_2,speed__time_reversal_asymmetry_statistic__lag_3,speed__value_count__value_0,speed__value_count__value_1,speed__

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics

In [ ]:
for i in range(2,20):
  cluster = AgglomerativeClustering(n_clusters=i, affinity='euclidean', linkage='ward')
  cluster.fit_predict(df)
  labels = cluster.labels_
  print("i:{} score: {}".format(i,metrics.silhouette_score(df, labels, metric='euclidean')))

i:2 score: 0.4722522149537888
i:3 score: 0.49651997021242883
i:4 score: 0.5149735044886968
i:5 score: 0.5345023961045773
i:6 score: 0.5114060757930128
i:7 score: 0.49644567060219263
i:8 score: 0.45219808519702837
i:9 score: 0.4535749646433999
i:10 score: 0.4521527865825605
i:11 score: 0.4405140387883338
i:12 score: 0.43336517230774657
i:13 score: 0.4070112941763019
i:14 score: 0.39156870516178355
i:15 score: 0.3859505771110975
i:16 score: 0.37876240079428075
i:17 score: 0.37672168975971054
i:18 score: 0.38208246274937574
i:19 score: 0.38312992913277666


In [ ]:
cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
cluster.fit_predict(df)
labels = cluster.labels_

In [ ]:
from collections import defaultdict
import plotly.express as px

In [ ]:
clusters = defaultdict(list)
cols = files
for i,val in enumerate(labels):
  clusters[val].append(cols[i])
for cluster in clusters:
  print("cluster:{} len:{}".format(cluster,len(clusters[cluster])))

cluster:1 len:372
cluster:3 len:228
cluster:4 len:130
cluster:0 len:482
cluster:2 len:254


In [ ]:
path = 'drive/MyDrive/Metadata/Coordinates_2017.csv'
df = pd.read_csv(path)
df["cluster"] = -1
for i in range(len(df)):
  file_name =  'drive/MyDrive/2017_v1/{}_{}_2017.csv'.format(int(df.loc[i]["vSegID"]),int(df.loc[i]["vSegDir"]))
  for cluster in clusters:
    if file_name in clusters[cluster]:
      df.iloc[i] = [int(df.loc[i]["vSegID"]), int(df.loc[i]["vSegDir"]),
                   df.loc[i]["longitude"], df.loc[i]["latitude"],
                   cluster]
      break

df.to_csv("drive/MyDrive/SimilarityScores/Coordinates/2017/Model_1_2017.csv")
path = "drive/MyDrive/SimilarityScores/Coordinates/2017/Model_1_2017.csv"
df = pd.read_csv(path)
df = df.drop("Unnamed: 0",axis=1)
df["cluster"] = df["cluster"].astype(str)
fig = px.scatter_mapbox(df, lon='latitude', lat='longitude', hover_name='vSegID', hover_data=['vSegDir',"cluster"],
                        color="cluster", zoom=4, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
